In [ ]:
class LogisticDNN(object):
    def __init__(self,n_features,n_labels):
        self.n_features = n_features
        self.n_labels = n_labels
        self.graph = tf.Graph()
        self.sess  = None
    
    def build(self,learning_rate):
        with self.graph.as_default():
            # Input
            self.train_set = tf.placeholder(tf.float32, shape=(None,self.n_features))
            self.train_lab = tf.placeholder(tf.int32  , shape=(None,self.n_labels))
            
            # Variables
            self.W1 = tf.Variable(
                    tf.truncated_normal( shape=(self.n_features, self.n_labels) ))
            self.b1 = tf.Variable(tf.zeros( shape=(self.n_labels) ))
            
            # Computation
            self.logits = tf.matmul(self.train_set,self.W1)+self.b1
            self.y = tf.nn.softmax(self.logits)
            self.loss = tf.reduce_mean(
                            tf.nn.softmax_cross_entropy_with_logits(labels=self.train_lab, logits=self.logits))

            
            # Optimalization
            self.train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.loss)
            
            # Prediction
            self.new_set = tf.placeholder(tf.float32, shape=(None,self.n_features))
            self.new_lab = tf.placeholder(tf.int32  , shape=(None,self.n_labels))
            
            self.new_logits = tf.matmul(self.new_set,self.W1)+self.b1
            self.new_y = tf.nn.softmax(self.new_logits)
            self.new_loss = tf.reduce_mean(
                                tf.nn.softmax_cross_entropy_with_logits(labels=self.new_lab, logits=self.new_logits))
    

    def getDenseLayer(self,input_layer,n_output):
        n_input = input_layer.shape[1]
        W = tf.Variable(
                tf.truncated_normal( shape=(n_input,n_output) ))
        b = tf.Variable(tf.zeros( shape=(n_output) ))
        s = tf.matmul(input_layer,W)+b
        return (W,b,s)
    
    def fit(self,X,y,learning_rate=0.5,epochs=10,validation_data=None,test_data=None):
        self.__structure(learning_rate)
        self.sess = tf.InteractiveSession(graph=self.graph)
        self.sess.run(tf.global_variables_initializer())
        for epoch in range(epochs):
            feed_dict = {self.train_set: X, self.train_lab: y}
            _, loss, predictions = self.sess.run([self.train_op, self.loss, self.y], feed_dict=feed_dict)
            acc = self.accuracy(predictions,y)
            print("Epoch %3d/%3d: "%(epoch+1,epochs), end='')
            print(" loss = %9.4f, acc = %3.2f%%" % ( loss,acc*100 ), end='')
            
            if validation_data:
                val_loss = self.evaluate(validation_data[0],validation_data[1])
                val_acc = self.accuracy(self.predict(validation_data[0]),validation_data[1])
                print(", val_loss = %9.4f, val_acc = %3.2f%%" % ( val_loss,val_acc*100 ), end='')
            
            print('')
            
        if test_data:
            test_acc = self.accuracy(self.predict(test_data[0]),test_data[1])
            print("test_acc = %3.2f%%" % (test_acc*100))
            
    def accuracy(self, predictions, labels):
        return (np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/predictions.shape[0])
    
    def predict(self,X):
        return self.sess.run(self.new_y, feed_dict={self.new_set: X})
    
    def evaluate(self,X,y):
        return self.sess.run(self.new_loss, feed_dict={self.new_set: X, self.new_lab: y})